# Homework 4: Petting a warg

Wargs do not make good pets. They are vicious creatures, populating Middle Earth, the world described by novels of John Ronald Reuel Tolkien. They tend to show up in the worst moment possible. They eat humans, hobbits, elves and wizards (when they can get them).

![A warg, getting ready for breakfast w:300px](figures/Gundabad_Wargs.jpg)

Your relationship with a warg can be in the following states:
```
SleepingWarg
AngryWarg
FuriousWarg
ApoplecticWarg
Safe
Sorry 
```

![tes](figures/WargStates.jpg)

Your actions are limited to petting a warg or striking it with your sword. The transitions are described in the following picture. The safe and sorry states are terminal, where no further actions can be taken. Landing into them has the reward +10 and -10 respectively. All other actions have a reward of -1. 

The discount factor is $\gamma=0.9$

![](figures/PetAWarg.jpg)


# How to solve this homework
The following problems you can solve either with the help of an LLM or by hand. 

* If you are solving by hand, make sure that you add sufficient comments to make sure that the code is understandable. 
* If you are solving using an LLM, add in form of comments
    * the LLM used (at the first use instance)
    * the prompt used to elicit the code
    * modifications that had to be done to the code 

For example:

```
# --- LLM used: ChatGPT 4.5
# --- LLM prompt
# Write a python class to encapsulate the least common multiple algorithm
# --- End of LLM prompt
```

The programming language should be Python.

## P1: MDP implementation 

Write a class to implement an MDP. Do not include value or policy iteration in the class.

In [15]:
class MDP:

    def __init__(self, states, terminal_states, actions, discount_factor):

        self.states = states
        self.start_state = states[0]
        self.actions = actions
        self.discount_factor = discount_factor
        self.terminal_states = terminal_states
        # --- LLM used: Claude Sonnet 4.5
        # --- LLM prompt: 
        # What is a way I can implement a set of transition functions without having to pass them into the class
        # to ensure I can store the probabiulity and reward in the same structure?
        self.transitions = {s: {a: [] for a in actions} for s in states}
        
    def add_transition(self, state, action, info: tuple[float, str, int]):
        probability, next_state, reward = info
        self.transitions[state][action].append((probability, next_state, reward))
    # --- End of LLM prompt

    def is_terminal(self, state):
        return (state in self.terminal_states)

    def get_transitions(self, state, action):
        return self.transitions[state][action]
    
    def get_actions(self, state):
        if self.is_terminal(state):
            return []
        return self.actions
    
    def get_discount_factor(self):
        return self.discount_factor
    
    def get_states(self):
        return self.states
    
    def get_terminal_states(self):
        return self.terminal_states

## P2: Warg as an MDP
Implement the WargPettingGame as an MDP using the implementation from above. 

In [16]:
def WargPettingGame_MDP():
    states = ['SleepingWarg', 'AngryWarg', 'FuriousWarg', 'ApoplecticWarg']
    terminal_states = ['Safe', 'Sorry']
    actions = ['Strike', 'Pet']
    discount_factor = 0.9

    game = MDP(states, terminal_states, actions, discount_factor)
    # SleepingWarg State
    game.add_transition('SleepingWarg', 'Strike', (1.0, 'AngryWarg', -1))
    game.add_transition('SleepingWarg', 'Pet', (0.95, 'AngryWarg', -1))
    game.add_transition('SleepingWarg', 'Pet', (0.05, 'Safe', 10))
    # AngryWarg State
    game.add_transition('AngryWarg', 'Strike', (1.0, 'FuriousWarg', -1))
    game.add_transition('AngryWarg', 'Pet', (1.0, 'Sorry', -10))
    # FuriousWarg State
    game.add_transition('FuriousWarg', 'Strike', (1.0, 'ApoplecticWarg', -1))
    game.add_transition('FuriousWarg', 'Pet', (1.0, 'Sorry', -10))
    # ApoplecticWarg
    game.add_transition('ApoplecticWarg', 'Strike', (0.8, 'Sorry', -10))
    game.add_transition('ApoplecticWarg', 'Strike', (0.2, 'Safe', 10))
    game.add_transition('ApoplecticWarg', 'Pet', (1.0, 'Sorry', -10))

    return game

## P3: Value iteration

Implement the value iteration as a separate function that uses this MDP implementation. 

In [17]:
def value_iteration(mdp: MDP, iterations=10000, convergence=0.0001):
    # Inilialize V for every state to 0.0
    V = {s: 0.0 for s in mdp.get_states()} 
    for terminal_state in mdp.get_terminal_states():
        V[terminal_state] = 0.0
    for i in range(iterations):
        V_next = V.copy() # V_k+1
        delta = 0 # Difference between iterations for convergence

        # --- LLM used: Claude Sonnet 4.5
        # --- LLM prompt
        # How can I loop through each of the possible transitions for a given action and compute the q value for 
        # value iteration. Please review these slides from my class to understand the bellman equation we used.

        # For each non-terminal state
        for state in mdp.get_states():

            # Compute Q*(s,a) for each action
            action_values = []
            for action in mdp.get_actions(state):
                q = 0

                # Q*(s,a) = Σ T(s,a,s') * [R(s,a,s') + γ * V_k(s')]
                transitions = mdp.get_transitions(state, action)
                for info in transitions:
                    probability, next_state, reward = info
                    q += probability * (reward + mdp.get_discount_factor() * V[next_state])

                action_values.append(q)

            # V_{k+1}(s) = max_a Q*(s,a) (Bellman update)
            if action_values:
                V_next[state] = max(action_values)
                delta = max(delta, abs(V_next[state] - V[state]))

        V = V_next
        # --- End of LLM prompt
        if delta < convergence:
            break

    return V

## P4: Using value iteration
Find the V* values of the WargPettingGame using the implementation above. Print out the V* values for each state in the form 
V(state) == number

In [18]:
mdp = WargPettingGame_MDP()
V_star = value_iteration(mdp)
for state, action in V_star.items():
    print(f"V({state}) = {action}")


V(SleepingWarg) = -6.2298
V(AngryWarg) = -6.760000000000001
V(FuriousWarg) = -6.4
V(ApoplecticWarg) = -6.0
V(Safe) = 0.0
V(Sorry) = 0.0


## P5:  Policy extraction

Find the policy $\pi(s)$ from the V values obtained in the previous step. Remember that you need to do one step of expectimax.
Print out the policy for each state, in a readable way. Eg. 
    pi(ApoplecticWarg) = Pet



In [19]:
def extract_policy(mdp: MDP, V: dict):
    policy = {}
    
    for state in mdp.get_states():
        policy[state] = None
            
        
        best_action = None
        best_value = float('-inf')
        
        # Try each action
        for action in mdp.get_actions(state):
            q_value = 0
            transitions = mdp.get_transitions(state, action)
            
            # Compute expected value for this action
            for prob, next_state, reward in transitions:
                q_value += prob * (reward + mdp.get_discount_factor() * V[next_state])
            
            # Update best action if this is better
            if q_value > best_value:
                best_value = q_value
                best_action = action
        
        policy[state] = best_action
    
    return policy

policy = extract_policy(mdp, V_star)
for state, action in policy.items():
    print(f"pi({state}) = {action}")

pi(SleepingWarg) = Pet
pi(AngryWarg) = Strike
pi(FuriousWarg) = Strike
pi(ApoplecticWarg) = Strike


## P6: Policy iteration
Implement policy iteration with the MDP as defined above as a separate function.
Apply it to the MDP defining the pet the warg game. 
Print out the resulting policy for each state, in a readable way.

In [20]:
# --- LLM used: Claude Sonnet 4.5
# --- LLM prompt: 
# Please help me produce policy iteration based on the previous MDP class provided. I have already implemented policy
# extraction with the fucntion extract_policy(mdp: MDP, V: dict) that you may call. Please set the max iterations attempt
# to 10000 and convergence to 0.0001 then print the results of the policy iteration function.
def policy_evaluation(mdp: MDP, policy: dict, convergence=0.0001):
    """
    Evaluate a fixed policy to find V^π.
    Similar to value_iteration but follows policy instead of taking max.
    """
    # Initialize V for all states
    V = {s: 0.0 for s in mdp.get_states()}
    for terminal_state in mdp.get_terminal_states():
        V[terminal_state] = 0.0
    
    for i in range(10000):  # Max iterations
        V_next = V.copy()
        delta = 0
        
        for state in mdp.get_states():
            if mdp.is_terminal(state):
                continue
            
            # KEY DIFFERENCE: Use policy's action, don't take max
            action = policy[state]
            if action is None:
                continue
            
            # Compute value for this specific action
            v = 0
            transitions = mdp.get_transitions(state, action)
            for prob, next_state, reward in transitions:
                v += prob * (reward + mdp.get_discount_factor() * V[next_state])
            
            V_next[state] = v
            delta = max(delta, abs(V_next[state] - V[state]))
        
        V = V_next
        
        if delta < convergence:
            break
    
    return V


def policy_iteration(mdp: MDP, max_iterations=10000):
    """Policy iteration using existing extract_policy function."""
    # Initialize with random policy
    policy = {}
    for state in mdp.get_states():
        actions = mdp.get_actions(state)
        policy[state] = actions[0] if actions else None
    
    for iteration in range(max_iterations):
        # Step 1: Evaluate current policy
        V = policy_evaluation(mdp, policy)
        
        # Step 2: Improve policy (reuse your extract_policy!)
        new_policy = extract_policy(mdp, V)
        
        # Check convergence
        if new_policy == policy:
            print(f"Policy iteration converged in {iteration + 1} iterations")
            break
        
        policy = new_policy
    
    return policy

# Run it
print("Policy Iteration Result:")
policy_iter_result = policy_iteration(mdp)
for state, action in policy_iter_result.items():
    print(f"π({state}) = {action}")

# --- End LLM prompt


Policy Iteration Result:
Policy iteration converged in 2 iterations
π(SleepingWarg) = Pet
π(AngryWarg) = Strike
π(FuriousWarg) = Strike
π(ApoplecticWarg) = Strike


## P7: Trajectory sampling
Implement a function that generates trajectories in the form of (s,a,r,s') tuples from the MDP for a specific policy. The trajectory ends when it reaches a terminal state. 

Generate 100 trajectories for a __random__ policy. 

In [21]:
import random
import numpy as np
# --- LLM used: Claude Sonnet 4.5
# --- LLM prompt:
# I need to implement trajectory sampling for an MDP. The function should generate trajectories 
# in the form of (state, action, reward, next_state) tuples by simulating the MDP. 
# The trajectory should start from the start_state and continue until reaching a terminal state.
# I need to handle two types of policies: a dictionary mapping states to actions, or 'random' 
# which chooses actions randomly. For stochastic transitions, sample the next state according 
# to the transition probabilities using numpy.random.choice.
# Also create a function to generate multiple trajectories.
def generate_trajectory(mdp: MDP, policy, start_state=None):
    """
    Generate a single trajectory following a policy until reaching terminal state.
    
    Args:
        mdp: MDP object
        policy: Either a dict mapping states to actions, or 'random' for random policy
        start_state: Starting state (if None, uses mdp.start_state)
    
    Returns:
        List of (state, action, reward, next_state) tuples
    """
    trajectory = []
    current_state = start_state if start_state else mdp.start_state
    
    while not mdp.is_terminal(current_state):
        # Choose action based on policy
        if policy == 'random':
            actions = mdp.get_actions(current_state)
            if not actions:
                break
            action = random.choice(actions)
        else:
            action = policy.get(current_state)
            if action is None:
                break
        
        # Get possible transitions for this state-action pair
        transitions = mdp.get_transitions(current_state, action)
        
        # Sample next state according to transition probabilities
        probs = [t[0] for t in transitions]
        next_states = [t[1] for t in transitions]
        rewards = [t[2] for t in transitions]
        
        # Sample based on probabilities
        idx = np.random.choice(len(transitions), p=probs)
        next_state = next_states[idx]
        reward = rewards[idx]
        
        # Record transition
        trajectory.append((current_state, action, reward, next_state))
        current_state = next_state
    
    return trajectory


def generate_trajectories(mdp: MDP, policy, num_trajectories=100, start_state=None):
    """Generate multiple trajectories."""
    trajectories = []
    for _ in range(num_trajectories):
        traj = generate_trajectory(mdp, policy, start_state)
        trajectories.append(traj)
    return trajectories
# --- End LLM prompt

# Generate 100 trajectories with random policy
print("\nGenerating 100 random trajectories...")
random_trajectories = generate_trajectories(mdp, 'random', num_trajectories=100)

print(f"Generated {len(random_trajectories)} trajectories")
print(f"Example trajectory 1 (length {len(random_trajectories[0])}):")
for step in random_trajectories[0]:
    state, action, reward, next_state = step
    print(f"  {state} --[{action}, r={reward}]--> {next_state}")


Generating 100 random trajectories...
Generated 100 trajectories
Example trajectory 1 (length 4):
  SleepingWarg --[Strike, r=-1]--> AngryWarg
  AngryWarg --[Strike, r=-1]--> FuriousWarg
  FuriousWarg --[Strike, r=-1]--> ApoplecticWarg
  ApoplecticWarg --[Pet, r=-10]--> Sorry


## P8: Implement Q-learning 

Create an implementation of Q-learning which takes the trajectory database and updates a Q-table.

In [22]:
from collections import defaultdict
# --- LLM used: Claude Sonnet 4.5
# --- LLM prompt:
# Implement Q-learning that takes a database of trajectories and updates a Q-table.
# The Q-table should be initialized with a default value for all state-action pairs.
# For each (state, action, reward, next_state) tuple in the trajectories, apply the Q-learning
# use the update rule for Q from the provided powerpoint slides.
# Use collections.defaultdict for the Q-table structure.
def q_learning(trajectories, mdp: MDP, alpha=0.1, initial_q=0.0):
    """
    Perform Q-learning on a database of trajectories.
    
    Args:
        trajectories: List of trajectories (each is list of (s,a,r,s') tuples)
        mdp: MDP object
        alpha: Learning rate
        initial_q: Initial Q-value for all state-action pairs
    
    Returns:
        Q-table as nested dictionary Q[state][action]
    """
    # Initialize Q-table
    Q = defaultdict(lambda: defaultdict(lambda: initial_q))
    
    # Initialize all state-action pairs explicitly
    for state in mdp.get_states():
        for action in mdp.get_actions(state):
            Q[state][action] = initial_q
    
    # Process all trajectories
    for trajectory in trajectories:
        for state, action, reward, next_state in trajectory:
            # Get max Q-value for next state
            if mdp.is_terminal(next_state):
                max_q_next = 0.0
            else:
                next_actions = mdp.get_actions(next_state)
                if next_actions:
                    max_q_next = max([Q[next_state][a] for a in next_actions])
                else:
                    max_q_next = 0.0
            
            # Q-learning update rule:
            # Q(s,a) ← Q(s,a) + α * [r + γ * max_a' Q(s',a') - Q(s,a)]
            Q[state][action] = Q[state][action] + alpha * (
                reward + mdp.get_discount_factor() * max_q_next - Q[state][action]
            )
    
    return Q

# --- End LLM prompt

## P9: Run Q-learning 

Run your implementation of Q-learning on the warg petting game. Print out the Q values in the form 

Q(state, action) = number


In [23]:
# --- LLM used: Claude Sonnet 4.5
# --- LLM prompt:
# Create a function to run Q-learning on the Warg MDP. The function should:
# 1. Generate multiple episodes of trajectories using a random policy
# 2. Run Q-learning on the collected trajectories
# 3. Print the resulting Q-values in the format "Q(state, action) = number"
# Use parameters: 100 trajectories per episode, 50 episodes, and learning rate alpha=0.1
def run_q_learning(mdp: MDP, num_trajectories=100, num_episodes=50, alpha=0.1):
    """
    Run Q-learning on the MDP with multiple episodes of trajectory generation.
    
    Args:
        mdp: MDP object
        num_trajectories: Number of trajectories per episode
        num_episodes: Number of episodes (iterations of trajectory generation + learning)
        alpha: Learning rate
    """
    print("\nRunning Q-Learning...")
    
    # Collect trajectories over multiple episodes
    all_trajectories = []
    for episode in range(num_episodes):
        # Generate trajectories using random policy
        trajectories = generate_trajectories(mdp, 'random', num_trajectories)
        all_trajectories.extend(trajectories)
    
    # Run Q-learning on all collected trajectories
    Q = q_learning(all_trajectories, mdp, alpha=alpha)
    
    # Print Q-values
    print("\nQ-values:")
    for state in mdp.get_states():
        for action in mdp.get_actions(state):
            print(f"Q({state}, {action}) = {Q[state][action]:.4f}")
    
    return Q

# Run Q-learning
Q_table = run_q_learning(mdp, num_trajectories=100, num_episodes=50, alpha=0.1)

# --- End LLM prompt


Running Q-Learning...

Q-values:
Q(SleepingWarg, Strike) = -9.0271
Q(SleepingWarg, Pet) = -8.3494
Q(AngryWarg, Strike) = -8.9077
Q(AngryWarg, Pet) = -10.0000
Q(FuriousWarg, Strike) = -8.8436
Q(FuriousWarg, Pet) = -10.0000
Q(ApoplecticWarg, Strike) = -9.0422
Q(ApoplecticWarg, Pet) = -10.0000


## P10: Policy implied by Q-values

Write a function that extracts a policy form q-values. 
Apply it to the Q-table obtained at P9. Print out the resulting policy in a readable way. 

In [24]:
# --- LLM used: Claude Sonnet 4.5
# --- LLM prompt:
# Write a function to extract a policy from Q-values. For each state, the policy should 
# select the action with the highest Q-value. Terminal states should have None as their action. 
# Print the resulting policy in a readable table format

def extract_policy_from_q(Q, mdp: MDP):
    """
    Extract policy from Q-values by choosing action with highest Q-value.
    
    Args:
        Q: Q-table (nested dict)
        mdp: MDP object
    
    Returns:
        Policy dictionary mapping states to actions
    """
    policy = {}
    
    for state in mdp.get_states():
        if mdp.is_terminal(state):
            policy[state] = None
        else:
            # Find action with maximum Q-value
            best_action = None
            best_value = float('-inf')
            
            for action in mdp.get_actions(state):
                if Q[state][action] > best_value:
                    best_value = Q[state][action]
                    best_action = action
            
            policy[state] = best_action
    
    return policy

# Extract and print policy from Q-learning
print("\nPolicy from Q-Learning:")
q_policy = extract_policy_from_q(Q_table, mdp)
for state, action in q_policy.items():
    if action is None:
        print(f"π({state}) = Terminal")
    else:
        print(f"π({state}) = {action}")

# Compare with value iteration policy
print("\nComparison:")
print("State               | Value Iteration | Q-Learning")
print("-" * 55)
for state in mdp.get_states():
    vi_action = policy.get(state, 'N/A')
    ql_action = q_policy.get(state, 'N/A')
    match = "✓" if vi_action == ql_action else "✗"
    print(f"{state:18} | {vi_action:15} | {ql_action:15} {match}")

# --- End LLM prompt


Policy from Q-Learning:
π(SleepingWarg) = Pet
π(AngryWarg) = Strike
π(FuriousWarg) = Strike
π(ApoplecticWarg) = Strike

Comparison:
State               | Value Iteration | Q-Learning
-------------------------------------------------------
SleepingWarg       | Pet             | Pet             ✓
AngryWarg          | Strike          | Strike          ✓
FuriousWarg        | Strike          | Strike          ✓
ApoplecticWarg     | Strike          | Strike          ✓
